In [1]:
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [2]:
groq_api_key = os.environ.get("GROQ_API_KEY")

In [4]:
tagging_prompt = ChatPromptTemplate.from_template("""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

In [15]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
You are given a passage. First, translate it to English if it is not already in English. Then extract the desired information according to the 'Classification' function.

Passage:
{input}

Provide the translated passage (if necessary) and then extract:
1. The sentiment of the text
2. How aggressive the text is on a scale from 1 to 10
3. The language the text is written in
"""
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")

# LLM
llm = ChatGroq(model="llama3-8b-8192", max_tokens=200).with_structured_output(Classification)

tagging_chain = tagging_prompt | llm
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"

try:
    result = tagging_chain.invoke({"input": inp})
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")


sentiment='positive' aggressiveness=2 language='Spanish'


In [16]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = tagging_chain.invoke({"input": inp})
res.dict()

{'sentiment': 'negative', 'aggressiveness': 8, 'language': 'Spanish'}

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [17]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
You are given a passage. First, translate it to English if it is not already in English. Then extract the desired information according to the 'Classification' function.

Passage:
{input}

Provide the translated passage (if necessary) and then extract:
1. The sentiment of the text
2. How aggressive the text is on a scale from 1 to 10
3. The language the text is written in
"""
)

In [18]:
llm = ChatGroq(model = "llama3-8b-8192", max_tokens=200).with_structured_output(Classification)

In [19]:
tagging_chain = tagging_prompt | llm

In [21]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

Classification(sentiment='positive', aggressiveness=2, language='Spanish')

In [22]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
tagging_chain.invoke({"input": inp})

Classification(sentiment='negative', aggressiveness=9, language='Spanish')

In [23]:
inp = "Weather is ok here, I can go outside without much more than a coat"
tagging_chain.invoke({"input": inp})

Classification(sentiment='positive', aggressiveness=2, language='English')